# Gallery Example: M/M/1-PS Queue (Processor Sharing)

This example demonstrates an M/M/1 queue with Processor Sharing scheduling:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Scheduling**: PS (Processor Sharing) - all jobs share the server equally
- **Capacity**: Infinite

Under PS, all jobs in the system receive equal share of the server's capacity, simulating time-sharing systems.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_ps(): Network {    """Create M/M/1-PS queueing model"""    model = Network("M/M/1-PS")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.PS)    sink = Sink(model, "mySink")        # Block 2: classes    oclass = OpenClass(model, "myClass")    source.setArrival(oclass, Exp(1))  # λ = 1    queue.setService(oclass, Exp(2))   # μ = 2, so ρ = λ/μ = 0.5        # Block 3: topology    P = model.initRoutingMatrix()    P.addRoute(oclass, source, queue, 1.0)    P.addRoute(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_mm1_ps()

## Theoretical Results for M/M/1-PS

For M/M/1-PS with λ=1, μ=2 (ρ=0.5):
- **Utilization**: ρ = 0.5 (same as FCFS)
- **Mean Queue Length**: L = ρ/(1-ρ) = 0.5/(1-0.5) = 1.0 (same as FCFS)
- **Mean Response Time**: W = 1/(μ-λ) = 1/(2-1) = 1.0 seconds (same as FCFS)
- **Throughput**: X = λ = 1.0

**Key Difference**: While mean performance is identical to FCFS, PS provides better perceived performance for short jobs and worse for long jobs.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare PS vs FCFS scheduling
println("\n=== PS vs FCFS Comparison ===")
// Create equivalent FCFS model
fun create_mm1_fcfs(): Network {
    val model_fcfs = Network("M/M/1-FCFS")
    val source = Source(model_fcfs, "mySource")
    val queue = Queue(model_fcfs, "myQueue", SchedStrategy.FCFS)
    val sink = Sink(model_fcfs, "mySink")
    
    val oclass = OpenClass(model_fcfs, "myClass")
    source.setArrival(oclass, Exp(1))
    queue.setService(oclass, Exp(2))
    
    val P = model_fcfs.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_fcfs.link(P)
    
    return model_fcfs
// Solve both models
val model_fcfs = create_mm1_fcfs()
val solver_ps = MVA(model)
val solver_fcfs = MVA(model_fcfs)

val avg_table_ps = solver_ps.avgTable
val avg_table_fcfs = solver_fcfs.avgTable

println("M/M/1-PS Results:")
println(avg_table_ps)

println("\nM/M/1-FCFS Results:")
println(avg_table_fcfs)

println("\nObservation: Mean performance metrics (utilization, queue length, response time)")
println("are identical between PS and FCFS for single-class M/M/1 systems.")
println("The difference lies in the service time variance and job-size fairness.")

In [ ]:
// Analyze PS behavior under different loads
println("\n=== Load Analysis for PS ===")
val load_factors = [0.3, 0.5, 0.7, 0.9, 0.95]

println("Load Factor | Utilization | Queue Length | Response Time")
println("-" * 60)

for rho in load_factors:
// Create model with specified load factor
    val model_load = Network(f"M/M/1-PS-rho{rho}")
    val source = Source(model_load, "Source")
    val queue = Queue(model_load, "Queue", SchedStrategy.PS)
    val sink = Sink(model_load, "Sink")
    
    val oclass = OpenClass(model_load, "Class")
    source.setArrival(oclass, Exp(1))  # λ = 1
    queue.setService(oclass, Exp(1/rho))  # μ = 1/ρ, so ρ = λ/(1/ρ) = ρ
    
    val P = model_load.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_load.link(P)
// Solve and extract metrics
    val solver = MVA(model_load)
    val avg_table = solver.avgTable
    
    val util = float(avg_table.iloc[1, 1])  # Queue utilization
    val queue_length = float(avg_table.iloc[1, 3])  # Queue length
    val resp_time = float(avg_table.iloc[1, 2])  # Response time
    
    println(f"    {rho:4.2f}    |    {util:5.3f}    |     {queue_length:5.3f}     |     {resp_time:6.3f}")